<center><img src="https://i.imgur.com/zRrFdsf.png" width="700"></center>


# Data Cleaning


Having data does not always allow you to produce some analytics right away. There is often a lot of pre processing to be done.

This material is about **Cleaning**: making sure each cell has a value that could be used in your coming procedures. There are always some _impurities_ that do not allow the computer to recognize the data correctly, i.e. _commas_ instead of _periods_ and viceversa, the presence of unneeded _blanks_, irrelevant symbols (dollar, euro symbols), or non-standard symbols to represent missing values.

I will use two approaches. The first one is the smart use of regular expressions (**regex**), and the second one a **divide and conquer** strategy.

# REGEX VERSUS DIVIDE AND CONQUER

Imagine that you request the age people in an online form. Sometimes you run into answers with issues like these:

- "It is:24"
- "It is: 44"
- "It is54"
- "64 it is"
- "I am twenty"
- "The 10th I turn 21"
- "I am 15 years old"
- "~20"

From the above examples, you are interested in the _age_, nothing else. The first two cases are _relatively_ easy to solve with divide and conquer, as you see a character that helps:  

In [ ]:
case1="It is:24"
case2="It is: 34"
# try 1
case1.split(':')[1]

In [ ]:
#try 2:
case2.split(':')[1]

Split broke the string using ":" and produced a _list_.  The number will be the second element. However, in _case2_ you got an extra space. You need to think about a general rule, so maybe this is better:

In [ ]:
case1.split(':')[1].strip()

In [ ]:
case2.split(':')[1].strip()

Using _strip()_ gets rid of the spaces around the string.  Notice _strip()_ and _split()_ are functions in **base Python**. Pandas has its **own** functions.

You can use the divide and conquer as long as every string you find follows the same pattern. Imagine those values make a column in a data frame:

In [ ]:
import pandas as pd

ages=["It is:24","It is: 44","It is54",
      "64 it is","I am twenty","The 10th I turn 21",
      "I am 15 years old","~20"]

someData=pd.DataFrame({'age':ages})
someData

Now, let's use Pandas **own** strip and split:

In [ ]:
someData.age.str.split(':')

Or alternatively:

In [ ]:
someData.age.str.split(':',expand=True)

Notice the use of _expand_. This allows that each element in the list goes to a column. However, as there is **no consistent pattern** (location of the symbol ":"), you do not get a good result. The situation requires the **REGEX** approach.

Using regular expressions is a great way to go when there is no pattern to apply the previous strategy; however, it takes time to learn how to build a regular expression that will serve all your especific  needs in a particular situation.

In general, you need to **explore** few *regex pattern*s before deciding what to use. I recommend using **contains()** for that:

In [ ]:
# do each cell has a character that is not a number? (\D)
someData.age.str.contains(pat=r'\D',
                          regex=True)

In [ ]:
# do each cell has a number character? (\d)
someData.age.str.contains(pat=r'\d',regex=True)

In [ ]:
# filter
someData[~someData.age.str.contains(pat=r'\d',regex=True)]

In [ ]:
# is there a cell where you have
# symbols beyond [^ ] alphanumeric (\w) or spaces (\s)?
someData.age[someData.age.str.contains(pat=r'[^\w\s]',regex=True)]

In [ ]:
# what happens if I erase all non numbers (\D)?
someData.age.str.replace(pat=r'\D',repl='',regex=True)

In [ ]:
# what happens if I extract consecutive numeric characters (\d+) ?
someData.age.str.extract(pat=r'(\d+)',expand=True)

In [ ]:
# what happens if I erase all
# numbers (\d+) followed by a texts [[a-z]+] ?
someData.age.str.replace(pat=r'\d+[a-z]+',
                         repl='',
                         regex=True)

In [ ]:
# so:
someData.age.str.replace(pat=r'\d+[a-z]+',
                         repl='',
                         regex=True).\
             str.extract(pat=r'(\d+)',expand=True)

In [ ]:
# using '|' ("or")
# ^ beginning of string
# $ end of the string
someData.age.str.extract(pat=r'(^\d+|\d+$|\s\d+\s)',
                         expand=True)

Let me use both results:

In [ ]:
someData['age1']=someData.age.str.replace(pat=r'\d+[a-z]+',
                                          repl='',
                                          regex=True).\
                                str.extract(pat=r'(\d+)',expand=True)

someData['age2']=someData.age.str.extract(pat=r'(^\d+|\d+$|\s\d+\s)',
                         expand=True)

In [ ]:
# we have
someData

In [ ]:
#checking data type
someData.info()

In [ ]:
# they are same....right?
someData['age1'].to_list()==someData['age2'].to_list()

In [ ]:
# where is the difference?
someData['age1']==someData['age2']

In [ ]:
# the same in both:
set(someData['age1']) & set(someData['age2'])

In [ ]:
#different in both
set(someData['age1']) ^ set(someData['age2'])

The last results tells you why they are not the same. Notice that seeing a pandas Series as a list reveals the details:

In [ ]:
someData['age1'].to_list()

In [ ]:
someData['age2'].to_list()

In [ ]:
someData['age2'].str.strip().to_list()